In [1]:
import pandas as pd
import xml.etree.ElementTree as ET
import os
from xml.etree.ElementTree import ElementTree
from itertools import chain
from lxml import etree
import re

In [2]:
# dir_path = r'/home/trishalaswain/data_for_analysis/SCP/Prod/2474/SCP2474/test_xml_read/'
dir_path = r'/home/trishalaswain/data_for_analysis/SCP/Prod/2474/SCP2474/SCP_2474_84_Final/'
res = []
for path in os.listdir(dir_path):
    
    if os.path.isfile(os.path.join(dir_path, path)) and str(path).endswith("pcs.xml"):
        res.append(dir_path + path)


In [3]:
len(res)

758

In [4]:
df = pd.DataFrame()
df['account'] = None
df['problem'] = None
df['cui'] = None

In [5]:
from collections import defaultdict
problem_dict = {}
df_final = pd.DataFrame()
for item in res:
    
    last_occ = item.rfind('/')
    # print(item[last_occ+1:])
    data = etree.parse(item)
    index = 0
    
    problem_section_list = ['CLINICAL IMPRESSION','Final diagnoses','diagnosis','IMPRESSION']
    problem_section_list.append('Differential Diagnosis:')
    problem_section_list.append('The differential diagnosis includes, but is not limited to:')
    problem_section_list.append('Diagnoses')
    for i in problem_section_list:
        
        for problem in data.xpath("//Section[@value='{}']/Paragraph/Sentence/RelationSet/problem".format(i)):
            
            # namespaces={"re": "http://exslt.org/regular-expressions"}
            # doc.xpath("//a[re:match(text(), 'some text')]",
            # print(problem.attrib['cui'])
            # print(problem.attrib['value'])
            
            df.at[index, 'account'] = item[last_occ+1:]
            df.at[index, 'problem'] = problem.attrib['value']
            df.at[index, 'cui'] = problem.attrib['cui']
            index += 1
            df_final = pd.concat([df,df_final])


In [6]:
df_final = df_final.reset_index(drop = True)

In [7]:
df_final

,account,problem,cui
0,86544747_99285_99284_pcs.xml,URI,C0041912;C3272713;C1548524
1,86544747_99285_99284_pcs.xml,cough,S00003732;C0010200;C0687152;C1961131
2,86544747_99285_99284_pcs.xml,congestion,C0700148
3,86544747_99285_99284_pcs.xml,Pharyngitis,C0031350
4,86544747_99285_99284_pcs.xml,URI,C0041912;C3272713;C1548524
...,...,...,...
339069,86992359_99285_99284_pcs.xml,Leg pain_SEP_left,S20004997;C0564822
339070,86992359_99285_99284_pcs.xml,Leg pain,S00022255;S00013031;C0023222;C0839480
339071,86992359_99285_99284_pcs.xml,pain,C0030193;C1557397;C1882272;C0242936
339072,86992359_99285_99284_pcs.xml,Leg pain,S00022255;S00013031;C0023222;C0839480


In [8]:
df2 = pd.DataFrame(df_final.problem.value_counts())

In [9]:
df3 = pd.DataFrame(df_final.cui.spl.value_counts())

AttributeError: 'Series' object has no attribute 'spl'

In [ ]:
df_final_2 = df_final.groupby(['problem'],).agg(',  '.join)